## CS431/631 Big Data Infrastructure
### Winter 2018 - Assignment 4
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** _Yangjie Zhou_
* **ID:** _20744733_

---
#### Overview
For this assignment, you will be using Python and Spark to perform some simple analyses on relational (tabular) data.  You will use Spark to read tabular data from files and then answer simple queries about the data in those tables.

In addition to Python and Spark, you will need to use a little bit of SQL.  If you are already familiar with SQL, great.  If not, you will want to spend a short amount of time getting familiar with the basics.   Type "SQL tutorial" into your favorite search engine, and you will find many examples of text, interactive and video tutorials.   As a simple starting point, you might also want to look at [these slides](https://cs.uwaterloo.ca/~kmsalem/courses/cs743/F14/slides/sql.pdf), which give a short introduction to SQL.   Even this is much more than you will need for this assignment.

You will be working with tabular data based on the schema for the TPC-H benchmark, which is a standard test used to measure the performance of relational database systems.   The schema defines the tables that exist in the database, the information in each table, and relationships between information in one table and information in another.   The TPC-H schema models business information:  customers, orders, parts, suppliers, and so on.   You can find a diagram illustrating the TPC-H schema in the lecture notes.   You can also find it on page 13 of the [TPC-H benchmark specification](http://www.tpc.org/TPC_Documents_Current_Versions/pdf/tpc-h_v2.17.3.pdf).   The TPC-H schema is important for this assignment, so make sure that you keep this schema handy.

Finally, for this assignment you will be using Spark in a slightly different way than you have used it so far.  For previous assignments, you have used the original Spark RDD interface.   For this assignment, you will be using the newer Spark interface, which is based on DataFrames.   DataFrames are RDDs in which each element is constrained to have the same structure.   You can think of a DataFrame like a table in a relational database.   Each element of the DataFrame is a row or record in the table.   All records have the same structure.   There is a programming guide for Spark DataFrames [here](https://spark.apache.org/docs/latest/sql-programming-guide.html).  Start with that. There is also a [more detailed guide to the full programming interface](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html).

---
#### Getting Started
To get started, let's initialize Spark, load a couple of the TPC-H tables, and run some simple queries.
First, as always, we need to tell the Python interpreter where to find Spark, so run `findspark.init`

In [1]:
import findspark
findspark.init("/u/cs451/packages/spark")

Next, we launch Spark.  When you used the RDD interface for previous assignments, you created a `SparkContext` when you launched Spark.   To use Spark SQL and the DataFrame interface, you instead create a `SparkSession`.   You do that as shown in the next cell (run it!).    

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("YourTest").master("local[2]").getOrCreate()

Next, let's create DataFrames from the TPC-H data files.  We have installed the TPC-H data files in the directory `/u/cs451/data/TPC-H-0.1-TXT/`.   There is one file for each table in the TPC-H database, e.g., `nation.tbl` for the TPC-H Nation table, `customer.tbl` for the TPC-H Customer table, and so on.    These are plain text csv files, with the character | used as a field separator.

Create a Spark DataFrame corresponding to the TPC-H Nation table by loading the data from the `nation.tbl` file.   Run the code in the next cell to do this.   After you run this code, `nation_raw` will refer to your new Spark DataFrame.   The Spark `show()` method will display a small (default 20) number of elements from the DataFrame, so that you can inspect them. 

In [3]:
nation_raw = spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/nation.tbl",sep='|',inferSchema=True)
nation_raw.show()

+---+----------+---+--------------------+----+
|_c0|       _c1|_c2|                 _c3| _c4|
+---+----------+---+--------------------+----+
|  0|   ALGERIA|  0| haggle. carefull...|null|
|  1| ARGENTINA|  1|al foxes promise ...|null|
|  2|    BRAZIL|  1|y alongside of th...|null|
|  3|    CANADA|  1|eas hang ironic, ...|null|
|  4|     EGYPT|  4|y above the caref...|null|
|  5|  ETHIOPIA|  0|ven packages wake...|null|
|  6|    FRANCE|  3|refully final req...|null|
|  7|   GERMANY|  3|l platelets. regu...|null|
|  8|     INDIA|  2|ss excuses cajole...|null|
|  9| INDONESIA|  2| slyly express as...|null|
| 10|      IRAN|  4|efully alongside ...|null|
| 11|      IRAQ|  4|nic deposits boos...|null|
| 12|     JAPAN|  2|ously. final, exp...|null|
| 13|    JORDAN|  4|ic deposits are b...|null|
| 14|     KENYA|  0| pending excuses ...|null|
| 15|   MOROCCO|  0|rns. blithely bol...|null|
| 16|MOZAMBIQUE|  0|s. ironic, unusua...|null|
| 17|      PERU|  1|platelets. blithe...|null|
| 18|     CHI

Now you have a DataFrame to work with.   The columns of the DataFrame correspond to the fields of the TPC-H Nation table, so have a look at the TPC-H schema diagram to see what you are dealing with.   Column c0 is the NATIONKEY, column c1 is the NAME, c2 is the REGIONKEY, and so on.   Since this is a synthetic database, you'll notice that the data in some of the fields (like the COMMENT field) consists of random words.   That's fine.   You can also ask Spark to tell you about the type of data in each column:

In [4]:
nation_raw.dtypes

[('_c0', 'int'),
 ('_c1', 'string'),
 ('_c2', 'int'),
 ('_c3', 'string'),
 ('_c4', 'string')]

Before going on, let's clean this DataFrame up a bit, to make it easier to use.   First, let's assign names to the columns, so that we can remember what information each column holds.   Second, you'll notice that Spark has created an extra final column (filled with `null` values) because each line in the input file ends with a separator character (|).  Let's drop that final column, since we don't need it.   Run the following code to do this:

In [5]:
nation = nation_raw.toDF('NationKey','Name','RegionKey','Comment','extra').drop('extra').cache()
nation.show()

+---------+----------+---------+--------------------+
|NationKey|      Name|RegionKey|             Comment|
+---------+----------+---------+--------------------+
|        0|   ALGERIA|        0| haggle. carefull...|
|        1| ARGENTINA|        1|al foxes promise ...|
|        2|    BRAZIL|        1|y alongside of th...|
|        3|    CANADA|        1|eas hang ironic, ...|
|        4|     EGYPT|        4|y above the caref...|
|        5|  ETHIOPIA|        0|ven packages wake...|
|        6|    FRANCE|        3|refully final req...|
|        7|   GERMANY|        3|l platelets. regu...|
|        8|     INDIA|        2|ss excuses cajole...|
|        9| INDONESIA|        2| slyly express as...|
|       10|      IRAN|        4|efully alongside ...|
|       11|      IRAQ|        4|nic deposits boos...|
|       12|     JAPAN|        2|ously. final, exp...|
|       13|    JORDAN|        4|ic deposits are b...|
|       14|     KENYA|        0| pending excuses ...|
|       15|   MOROCCO|      

This style of code should look familar to you.  We started with the `nation_raw` DataFrame and applied a series of DataFrame operations (`toDF`, `drop`, and `cache`).   This is just like the RDD interface, except now we are applying DataFrame operations to DataFrames, instead of RDD operations to RDDs.

Next, let's load up the TPC-H Supplier table, and then try performing some queries:

In [6]:
supplier_raw = spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/supplier.tbl",sep='|',inferSchema=True).drop("_c7")
supplier = supplier_raw.toDF("SuppKey","Name","Address","NationKey","Phone","AcctBal","Comment").cache()
supplier.show()

+-------+------------------+--------------------+---------+---------------+-------+--------------------+
|SuppKey|              Name|             Address|NationKey|          Phone|AcctBal|             Comment|
+-------+------------------+--------------------+---------+---------------+-------+--------------------+
|      1|Supplier#000000001| N kD4on9OM Ipw3,...|       17|27-918-335-1736|5755.94|each slyly above ...|
|      2|Supplier#000000002|89eJ5ksX3ImxJQBvx...|        5|15-679-861-2259|4032.68| slyly bold instr...|
|      3|Supplier#000000003|q1,G3Pj6OjIuUYfUo...|        1|11-383-516-1199| 4192.4|blithely silent r...|
|      4|Supplier#000000004|Bk7ah4CK8SYQTepEm...|       15|25-843-787-7479|4641.08|riously even requ...|
|      5|Supplier#000000005|   Gcdm2rJRzl5qlTVzc|       11|21-151-690-3663|-283.84|. slyly regular p...|
|      6|Supplier#000000006|        tQxuVm7s7CnK|       14|24-696-997-4969|1365.79|final accounts. r...|
|      7|Supplier#000000007|s,4TicNGB4uO6PaSq...|      

---
#### Writing Queries
There are two equivalent ways of writing queries over Spark DataFrames.   The first way is to assign a "view name" to the DataFrame, and then write SQL queries referring to those view names using the `sql` operation.  

The code below gives the view names "nation" and "supplier" to the two DataFrames we've already created.

In [7]:
supplier.createOrReplaceTempView("supplier")
nation.createOrReplaceTempView("nation")

Now, we can write SQL queries that refer to the "supplier" and "nation" views as tables.   For example, suppose we want to see the names and addresses of suppliers who have account balances above 9900.00:

In [8]:
q1_result = spark.sql("select Name, Address, AcctBal from supplier where AcctBal > 9900.00")
q1_result.show()

+------------------+--------------------+-------+
|              Name|             Address|AcctBal|
+------------------+--------------------+-------+
|Supplier#000000049|     Nvq 6macF4GtJvz|9915.24|
|Supplier#000000234|iMrk7HUD87at3IIh4rBi| 9957.0|
|Supplier#000000693|S,mnHfsroFOVieQGd...|9956.55|
|Supplier#000000855|ekQwhb9fh5VGIvMBJ...|9964.88|
|Supplier#000000892|j6prA4M3sX9a9xHem...|9993.46|
+------------------+--------------------+-------+



In the example above, the `sql` command runs the SQL query against the supplier table.   It returns the query result as a new DataFrame, which `q1_result` refers to.

Instead of writing your queries in SQL and running them using `sql`, it is possible to do the same thing by applying a sequence of DataFrame operations to the input DataFrames, as you did when you were using the RDD interface in the previous assignments.    For example, to answer the same query that we just answered using SQL, we can do the following:

In [9]:
q1_resultB = supplier.filter("AcctBal > 9900.00").select('Name','Address','AcctBal')
q1_resultB.show()

+------------------+--------------------+-------+
|              Name|             Address|AcctBal|
+------------------+--------------------+-------+
|Supplier#000000049|     Nvq 6macF4GtJvz|9915.24|
|Supplier#000000234|iMrk7HUD87at3IIh4rBi| 9957.0|
|Supplier#000000693|S,mnHfsroFOVieQGd...|9956.55|
|Supplier#000000855|ekQwhb9fh5VGIvMBJ...|9964.88|
|Supplier#000000892|j6prA4M3sX9a9xHem...|9993.46|
+------------------+--------------------+-------+



Both methods should give the same result.   Internally, Spark handles both similarly.   For this assignment, you'll be asked to try out both methods.

Now it is time for you to try writing your own queries.

---
#### Question 1 (2/25 marks)
In the cell below, write a query that will return the ORDERKEY, ORDERDATE, and TOTALPRICE of the five orders with the highest TOTALPRICE.   Express your query in SQL, and use `sql` to execute it.

In [10]:
# Your solution to Question 1 here
orders_raw=spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/orders.tbl",sep="|",inferSchema=True)
orders = orders_raw.toDF("OrderKey","CustKey","OrderStatus",
                         "TotalPrice","OrderDate","Order-Priority",
                         "Clerk","Ship-Priority","Comment","extra").drop("extra").cache()
orders.createOrReplaceTempView("orders")

q1_result = spark.sql("SELECT OrderKey, OrderDate, TotalPrice FROM orders ORDER BY TotalPrice DESC LIMIT 5")
q1_result.show()


+--------+--------------------+----------+
|OrderKey|           OrderDate|TotalPrice|
+--------+--------------------+----------+
|  279812|1994-02-19 00:00:...| 479129.21|
|  370726|1996-09-29 00:00:...|  460099.4|
|   66659|1993-10-15 00:00:...| 458396.42|
|  253639|1998-01-23 00:00:...| 456532.89|
|  502886|1994-04-12 00:00:...| 456423.88|
+--------+--------------------+----------+



---
#### Question 2 (2/25 marks)
In the cell below, answer the same query you answered in Question 1, but this time do not use the `sql` method.

In [11]:
# Your solution to Question 2 here
q2_result=orders.select("OrderKey", "OrderDate", "TotalPrice")\
                .orderBy("TotalPrice",ascending=False)\
                .limit(5)
q2_result.show()

+--------+--------------------+----------+
|OrderKey|           OrderDate|TotalPrice|
+--------+--------------------+----------+
|  279812|1994-02-19 00:00:...| 479129.21|
|  370726|1996-09-29 00:00:...|  460099.4|
|   66659|1993-10-15 00:00:...| 458396.42|
|  253639|1998-01-23 00:00:...| 456532.89|
|  502886|1994-04-12 00:00:...| 456423.88|
+--------+--------------------+----------+



---
#### Question 3 (3/25 marks)
In the cell below, write code that will prompt for a Customer key, and then return the customer's name as well as the ORDERDATE and TOTALPRICE of that customer's most recent order.   Express the query in SQL, and use `sql` to execute it.   You will need to use information from multiple tables to generate your answer.

In [12]:
# Your solution to Question 3 here
# 用户名字（在用户table中）
# 订单时间和总价（在订单table中，更具体的应该在排序过的用户的订单中）
customer_raw=spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/customer.tbl",sep="|",inferSchema=True)
customer = customer_raw.toDF("CustKey","Name","Address","NationKey","Phone",
                             "Acctbal","Mktsegment","Comment","extra").drop("extra").cache()
customer.createOrReplaceTempView("customer")

while True:
    try:
        customer_key=input("input the customer key(press Enter to quit):")
        if len(customer_key)==0:
            break
        customer_key=int(customer_key)
        if customer_key<=0 or customer_key>15000:
            continue
    except  ValueError:
        print("customer key must be a positive integer ranging from 1 to 15000!")
        continue
    ## using the subquery to speed up
    q3_result=spark.sql("SELECT c.Name,o.OrderDate,o.TotalPrice\
                        FROM orders o INNER JOIN (select CustKey,Name from customer where CustKey={0}) c \
                        ON c.CustKey=o.CustKey \
                        ORDER BY OrderDate DESC\
                        LIMIT 1".format(customer_key))
    q3_result.show()

input the customer key(press Enter to quit):1
+------------------+--------------------+----------+
|              Name|           OrderDate|TotalPrice|
+------------------+--------------------+----------+
|Customer#000000001|1997-03-04 00:00:...| 268835.44|
+------------------+--------------------+----------+

input the customer key(press Enter to quit):2
+------------------+--------------------+----------+
|              Name|           OrderDate|TotalPrice|
+------------------+--------------------+----------+
|Customer#000000002|1998-05-22 00:00:...| 122500.55|
+------------------+--------------------+----------+

input the customer key(press Enter to quit):3
+----+---------+----------+
|Name|OrderDate|TotalPrice|
+----+---------+----------+
+----+---------+----------+

input the customer key(press Enter to quit):


---
#### Question 4 (3/25 marks)
In the cell below, answer the same query you answered in Question 3, but this time do not use the `sql` method.

In [13]:
# Your solution to Question 4 here
while True:
    try:
        customer_key=input("input the customer key(Press Enter to quit):")
        if len(customer_key)==0:
            break
        customer_key=int(customer_key)
        if customer_key<=0 or customer_key>15000:
            continue
    except  ValueError:
        print("Customer key must be a positive integer ranging from 1 to 15000!")
        continue
    
    ## judge whether this customer have ever ordered
    if len(orders.filter(orders.CustKey==customer_key).collect())==0:
        print("This customer never have an order")
        continue
        
    orders_short=orders.select("CustKey","OrderDate","TotalPrice")
    q4_result=customer.select("CustKey","Name")\
                    .where(customer.CustKey==customer_key)\
                    .join(orders_short,'CustKey')\
                    .orderBy("OrderDate",ascending=False)\
                    .limit(1).drop('CustKey')\
                    .collect()
    print(q4_result[0])

input the customer key(Press Enter to quit):1
Row(Name='Customer#000000001', OrderDate=datetime.datetime(1997, 3, 4, 0, 0), TotalPrice=268835.44)
input the customer key(Press Enter to quit):2
Row(Name='Customer#000000002', OrderDate=datetime.datetime(1998, 5, 22, 0, 0), TotalPrice=122500.55)
input the customer key(Press Enter to quit):3
This customer never have an order
input the customer key(Press Enter to quit):


---
#### Question 5 (5/25 marks)
In the cell below, write code that will prompt for a Nation name.  (Assume that nation names are unique in the Nation table.)   Report the number of distinct parts supplied by suppliers that are located in the given nation.

You may answer this question with or without using `sql` - whichever you prefer.

In [14]:
# Your solution to Question 5 here
nation_raw=spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/nation.tbl",sep="|",inferSchema=True)
nation = nation_raw.toDF("NationKey","Name","RegionKey","Comment","extra").drop("extra").cache()
nation.createOrReplaceTempView("nation")

while True:
    nation_name=input("input the Nation name(Press Enter to quite):")
    if len(nation_name)==0:
        break
    q5_result=spark.sql("select count(s.NationKey) \
                        from supplier s\
                        where s.NationKey \
                        in (select NationKey from nation where nation.Name='{0}')".format(nation_name))
    q5_result.show()

input the Nation name(Press Enter to quite):INDIA
+----------------+
|count(NationKey)|
+----------------+
|              47|
+----------------+

input the Nation name(Press Enter to quite):CHINA
+----------------+
|count(NationKey)|
+----------------+
|              53|
+----------------+

input the Nation name(Press Enter to quite):


---
#### Question 6 (5/25 marks)
In the cell below, write code that will prompt for a BRAND, like those that appear in the Parts table.  Given the BRAND,
report, for each nation, a count of that nation's suppliers of parts having that brand.   Your output should be a table of nations and their supplier counts. Each supplier should be counted at most once in a nation's total, even if that supplier produces multiple parts with the given brand. 

You may answer this question with or without using `sql` - whichever you prefer.

In [15]:
# Your solution to Question 6 here
# 对输入的品牌，输出一个关于国家的table
# 
part_raw=spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/part.tbl",sep="|",inferSchema=True)
part = part_raw.toDF("PartKey","Name","Mfgr","Brand","Type","Size","Container",
                     "RetailPrice","Comment","extra").drop("extra").cache()
part.createOrReplaceTempView("part")

partsupp_raw=spark.read.csv("/u/cs451/data/TPC-H-0.1-TXT/partsupp.tbl",sep="|",inferSchema=True)
partsupp = partsupp_raw.toDF("PartKey","SuppKey","Availqty","SupplyCost","Comment","extra").drop("extra").cache()
partsupp.createOrReplaceTempView("partsupp")

while True:
    brand=input("input the Brand(like \"Brand#13\" Press Enter to quite):")
    if len(brand)==0:
        break
        
    country_count_table=part.select("Brand","PartKey")\
            .filter(part.Brand==brand)\
            .join(partsupp,"PartKey")\
            .select("SuppKey")\
            .join(supplier,"SuppKey")\
            .select("NationKey")\
            .groupBy("NationKey")\
            .count()\
            .join(nation,"NationKey")\
            .select("Name","count").collect()
            
    print("Country \t Count")
    for i in country_count_table:
        print(i[0],'\t',i[1])

input the Brand(like "Brand#13" Press Enter to quite):Brand#13
Country 	 Count
JAPAN 	 115
RUSSIA 	 161
ARGENTINA 	 115
JORDAN 	 91
FRANCE 	 118
MOZAMBIQUE 	 108
CANADA 	 129
SAUDI ARABIA 	 159
ETHIOPIA 	 117
ROMANIA 	 112
MOROCCO 	 120
PERU 	 135
INDONESIA 	 145
EGYPT 	 127
INDIA 	 141
UNITED KINGDOM 	 126
GERMANY 	 160
IRAN 	 132
UNITED STATES 	 133
VIETNAM 	 137
IRAQ 	 150
KENYA 	 120
BRAZIL 	 154
ALGERIA 	 122
CHINA 	 197
input the Brand(like "Brand#13" Press Enter to quite):


---
#### Question 7 (5/25 marks)
In the cell below, write code that will prompt for a Nation name.   Report, for each year, the total number of orders placed by customers from the specified Nation.

You may answer this question with or without using `sql` - whichever you prefer.

In [16]:
# Your solution to Question 7 here

while True:
    nation_name=input("input the Nation name(Press Enter to quite):")
    if len(nation_name)==0:
        break
    raw=spark.sql("SELECT from_unixtime(unix_timestamp(OrderDate),\"Y\") as year,CustKey from orders\
                                 WHERE orders.CustKey\
                                 in (SELECT CustKey FROM customer WHERE customer.NationKey \
                                 in (SELECT NationKey FROM nation WHERE nation.Name=\"{0}\"))".format(nation_name))
    num_orders_country=sorted(raw.groupBy('year').agg({'CustKey': 'count'}).collect())
    print("the total number of orders placed by customers from the",nation_name,"for each year")
    print("year \t number of orders")
    for i in num_orders_country:
        print(i[0],'\t',i[1])

input the Nation name(Press Enter to quite):INDIA
the total number of orders placed by customers from the INDIA for each year
year 	 number of orders
1992 	 848
1993 	 850
1994 	 913
1995 	 888
1996 	 796
1997 	 880
1998 	 552
input the Nation name(Press Enter to quite):CHINA
the total number of orders placed by customers from the CHINA for each year
year 	 number of orders
1992 	 917
1993 	 902
1994 	 994
1995 	 884
1996 	 948
1997 	 915
1998 	 581
input the Nation name(Press Enter to quite):


---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.